In [ ]:
!wget http://www.cs.virginia.edu/~skadron/lava/Rodinia/Packages/rodinia_3.1.tar.bz2
!tar -xf rodinia_3.1.tar.bz2

--2022-12-03 14:13:59--  http://www.cs.virginia.edu/~skadron/lava/Rodinia/Packages/rodinia_3.1.tar.bz2
Resolving www.cs.virginia.edu (www.cs.virginia.edu)... 128.143.67.11
Connecting to www.cs.virginia.edu (www.cs.virginia.edu)|128.143.67.11|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cs.virginia.edu/~skadron/lava/Rodinia/Packages/rodinia_3.1.tar.bz2 [following]
--2022-12-03 14:14:00--  https://www.cs.virginia.edu/~skadron/lava/Rodinia/Packages/rodinia_3.1.tar.bz2
Connecting to www.cs.virginia.edu (www.cs.virginia.edu)|128.143.67.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 360636246 (344M) [application/x-tar]
Saving to: ‘rodinia_3.1.tar.bz2’

rodinia_3.1.tar.bz2 100%[===================>] 343.93M  11.7MB/s    in 30s     

2022-12-03 14:14:30 (11.6 MB/s) - ‘rodinia_3.1.tar.bz2’ saved [360636246/360636246]



In [ ]:

f=open('./common/make.config', 'wt') 
f.write("""CUDA_DIR = /usr/local/cuda-11
CUDA_LIB_DIR = ${CUDA_DIR}/lib64
SDK_DIR = ${CUDA_DIR}/samples/""")

f.close()


In [ ]:
from pathlib import Path 
import subprocess
import itertools

BaseDir = Path('.').absolute()

Types = ['cuda', 'openmp']
problems = ['heartwall', 'bfs', 'nn']
for impl, problem in itertools.product(Types, problems):
  path = BaseDir / 'rodinia_3.1' / impl / problem
  print(f'making {impl} {problem}\n{path}')
  p = subprocess.run(f"cd {path}; make -B", shell=True, capture_output=True)
  print(p.stdout.decode('utf-8'))
  print(p.stderr.decode('utf-8'))

making cuda heartwall
/content/rodinia/rodinia_3.1/cuda/heartwall
nvcc   main.cu -I./AVI -c -O3
cd AVI; make;
make[1]: Entering directory '/content/rodinia/rodinia_3.1/cuda/heartwall/AVI'
gcc -c avimod.c
gcc -c avilib.c
make[1]: Leaving directory '/content/rodinia/rodinia_3.1/cuda/heartwall/AVI'
cd AVI; make;
make[1]: Entering directory '/content/rodinia/rodinia_3.1/cuda/heartwall/AVI'
gcc -c avimod.c
gcc -c avilib.c
make[1]: Leaving directory '/content/rodinia/rodinia_3.1/cuda/heartwall/AVI'
nvcc main.o ./AVI/avilib.o ./AVI/avimod.o -I/usr/local/cuda/include -lm -o heartwall

main.cu(63): warning: variable "c" was declared but never referenced

main.cu: In function ‘void write_data(char*, int, int, int, int*, int*, int, int*, int*)’:
main.cu:86:31: warning: too many arguments for format [-Wformat-extra-args]
      fprintf(fid, "\n--endo--\n",j);
                               ^
main.cu:95:30: warning: too many arguments for format [-Wformat-extra-args]
      fprintf(fid, "\n--epi--\n"

In [ ]:
runmap = {
    ('cuda', 'heartwall'): './heartwall ../../data/heartwall/test.avi 1 ',
    ('openmp', 'heartwall'): './heartwall ../../data/heartwall/test.avi 1 4 ',
    ('cuda', 'bfs'): './bfs ../../data/bfs/graph1MW_6.txt',
    ('openmp', 'bfs'): './bfs 4 ../../data/bfs/graph1MW_6.txt ',
    ('cuda', 'nn'): './nn filelist_4 -r 5 -lat 30 -lng 90 ',
    ('openmp', 'nn'): './nn filelist_4 5 30 90  ',
}

In [ ]:
import numpy

In [ ]:
from time import perf_counter

for problem, impl in itertools.product(problems, Types):
  path = BaseDir / 'rodinia_3.1' / impl / problem
  print(f'\n=======\nrunning {impl} {problem} @ {path}')
  N = 10
  times = []
  for _ in range(N):
    start = perf_counter()
    p = subprocess.run(f"cd {path}; {runmap[(impl, problem)]}", shell=True, capture_output=True)
    stop = perf_counter()
    times.append(stop - start)
  
  m = numpy.mean(times)
  s = numpy.std(times)

  print(f'average elapsed time: {m:.2f} ± {s:.2f} seconds')
  print('STDOUT')
  print(p.stdout.decode('utf-8'))
  print('STDERR')
  print(p.stderr.decode('utf-8'))


running cuda heartwall @ /content/rodinia/rodinia_3.1/cuda/heartwall
average elapsed time: 0.85 ± 0.05 seconds
STDOUT
WG size of kernel = 256 
frame progress: 0 

STDERR


running openmp heartwall @ /content/rodinia/rodinia_3.1/openmp/heartwall
average elapsed time: 0.03 ± 0.00 seconds
STDOUT
num of threads: 4
frame progress: 0 

STDERR


running cuda bfs @ /content/rodinia/rodinia_3.1/cuda/bfs
average elapsed time: 2.01 ± 0.31 seconds
STDOUT
Reading File
Read File
Copied Everything to GPU memory
Start traversing the tree
Kernel Executed 12 times
Result stored in result.txt

STDERR


running openmp bfs @ /content/rodinia/rodinia_3.1/openmp/bfs
average elapsed time: 1.28 ± 0.17 seconds
STDOUT
Reading File
Start traversing the tree
Compute time: 0.063577
Result stored in result.txt

STDERR


running cuda nn @ /content/rodinia/rodinia_3.1/cuda/nn
average elapsed time: 0.83 ± 0.00 seconds
STDOUT
1988 12 27  0 18 TONY       30.0  89.8  113   39 --> Distance=0.199997
1980 10 22 18  3 ISAAC 